# Basic usage

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)

## Training a classifier and making predictions

In [3]:
import numpy as np
from sklearn.datasets import make_classification

In [24]:
X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X, y = X.astype(np.float32), y.astype(np.int64)

In [25]:
X.shape, y.shape, y.mean()

((1000, 20), (1000,), 0.5)

### Definition of the pytorch classification module

In [26]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
            dropout=0.5,
    ):
        super(ClassifierModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin
        self.dropout = dropout

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(dropout)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X), dim=-1)
        return X

### Defining and training neural net classifier

In [27]:
from skorch import NeuralNetClassifier

In [28]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1
)

In [29]:
net.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6952       0.4800        0.6942  0.0158
      2        0.6790       0.5250        0.6827  0.0129
      3        0.6765       0.5550        0.6727  0.0135
      4        0.6568       0.5900        0.6634  0.0132
      5        0.6529       0.6400        0.6543  0.0134
      6        0.6375       0.6500        0.6483  0.0125
      7        0.6164       0.6700        0.6411  0.0140
      8        0.6208       0.6650        0.6347  0.0120
      9        0.6145       0.6750        0.6289  0.0130
     10        0.6055       0.6900        0.6214  0.0132
     11        0.5932       0.6800        0.6163  0.0145
     12        0.5789       0.6800        0.6115  0.0131
     13        0.5635       0.6800        0.6081  0.0135
     14        0.5701       0.6850        0.6076  0.0138
     15        0.5568       0.6950        0.6000  0.0141
     16        0.5638       0.7

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=20, out_features=10, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (dense1): Linear(in_features=10, out_features=10, bias=True)
    (output): Linear(in_features=10, out_features=2, bias=True)
  ),
)

In [30]:
y_pred = net.predict(X[:5])
y_pred

array([1, 0, 0, 0, 0])

In [31]:
y_proba = net.predict_proba(X[:5])
y_proba

array([[0.35712352, 0.64287645],
       [0.90555364, 0.09444635],
       [0.7276744 , 0.2723256 ],
       [0.9050409 , 0.09495904],
       [0.74524426, 0.2547557 ]], dtype=float32)